In [ ]:
# Import data

import pandas as pd
import numpy as np

train = pd.read_csv("tsd_trial.csv")

In [ ]:
train = train[6939:].reset_index()
train.drop('index',axis = 1,inplace=True)

In [ ]:
# Make a list of list with character offsets of dataset

spans = []

for i, row in train.iterrows():
    to_append_list = [int(x) for x in str(train["spans"].iloc[i])[1:len(train["spans"].iloc[i])-1].split(", ") if len(x) > 0]
    spans.append(to_append_list)

In [ ]:
# Group non-contiguous spans

import more_itertools as mit
span_grouped = []

for i in range(len(spans)):
    span_grouped.append([list(group) for group in mit.consecutive_groups(spans[i])])

In [ ]:
# Find starting and ending offset of non-contiguos offset

final = []

for i in range(len(span_grouped)):
    span_grouped_just_group_sentence = []
    for ranges in span_grouped[i]:
        span_grouped_just_group_groups = []
        #print(ranges)
        if len(ranges) > 0:
            span_grouped_just_group_groups = [ranges[0],ranges[-1]]
        
        span_grouped_just_group_sentence.append(span_grouped_just_group_groups)
        
    final.append(span_grouped_just_group_sentence)     

In [ ]:
# Import Ground Truth

with open('gold.txt', 'w',encoding="utf-8") as f:
    for i in range(len(final)):
        for span in final[i]:
            if len(final[i]):
                #print(final[i])
                f.write(f"{i}\t{span[0]}\t{span[1]}" + "\n")

In [ ]:
# Import Prediction

with open("dev/bert_io_dev.txt", 'r') as file1:
    lines = file1.readlines()

answer_token = []
answers_token = []

answer_pred = []
answers_pred = []

for line in lines:
    if not (line.isspace()):
        feats = line.strip().split()
        answer_token.append(feats[0])
        answer_pred.append(feats[1])
        
    else:
        answers_token.append(answer_token)
        answers_pred.append(answer_pred)
        answer_token= []
        answer_pred = []

In [ ]:
# Function for changing scheme from IO --> BILOU

def change_scheme_IO_BILOU(input_list):
    if len(input_list) == 0:
        return input_list
    elif len(input_list) == 1:
        if input_list[0] == "I":
            input_list[0] = "U"
        else:
            return input_list

    for i in range(len(input_list)):
        if (input_list[i] == "I") and (i != 0) and (i != (len(input_list)-1)) :
            if (input_list[i-1] == "O") and (input_list[i+1] == "O"):
                input_list[i] = "U"
            elif input_list[i-1] == "O":
                input_list[i] = "B"
            elif input_list[i+1] == "O":
                input_list[i] = "L"
        elif input_list[i] == "I" and (i == 0):
            if input_list[i+1] == "O":
                input_list[i] = "U"
            elif input_list[i+1] == "I":
                input_list[i] = "B"
        elif input_list[i] == "I" and (i == (len(input_list) - 1)):
            if input_list[i-1] == "O":
                input_list[i] = "U"
            elif (input_list[i-1] == "I") or (input_list[i-1] == "B"):
                input_list[i] = "L"
                
    return input_list

In [ ]:
# Function for changing scheme from BIO --> BILOU

def change_scheme_BIO_BILOU(input_list):
    if len(input_list) == 0:
        return input_list
    elif len(input_list) == 1:
        if input_list[0] == "B":
            input_list[0] = "U"
        else:
            return input_list

    for i in range(len(input_list)):
        if (input_list[i] == "I") and (i != 0) and (i != (len(input_list)-1)) :
            if input_list[i+1] == "O":
                input_list[i] = "L"
                
        elif input_list[i] == "I" and (i == (len(input_list) - 1)):
            if (input_list[i-1] == "I") or (input_list[i-1] == "B"):
                input_list[i] = "L"
                
        elif (input_list[i] == "B") and (i != 0) and (i != (len(input_list)-1)) :
            if (input_list[i-1] == "O") and (input_list[i+1] == "O"):
                input_list[i] = "U"
            elif (input_list[i-1] == "I") or (input_list[i-1] == "B"):
                input_list[i] = "L"
            
                
        elif input_list[i] == "B" and (i == 0):
            if input_list[i+1] == "O":
                input_list[i] = "U"
        
        elif input_list[i] == "B" and (i == (len(input_list) - 1)):
            if (input_list[i-1] == "O"):
                input_list[i] = "U"
            elif (input_list[i-1] == "I") or (input_list[i-1] == "B"):
                input_list[i] = "L"
                
    return input_list

In [ ]:
# Call the function

for i in range(len(answers_pred)):
    answers_pred[i] = change_scheme_IO_BILOU(answers_pred[i])

In [ ]:
# Function to get starting offset in case of B

def get_start(sentence,index,to_find):
    
    flag = False
    
    split_sentence = sentence.split(" ")
    
    m = 0
    n = 0
    
    while m < len(split_sentence):
        #print(m)
        if to_find in split_sentence[m]:

            n = 0
            while n < len(sentence):
        
                start = m
                #print(sentence)
                #print(n)
                #print(split_sentence[start])
                #print(sentence.find(split_sentence[start],n))
                actual_start = sentence.find(split_sentence[start],n)
                
                while sentence[actual_start:actual_start + len(to_find)] != to_find:
                    actual_start += 1
                
                if sentence[actual_start] == "'":
                    flag = True

                
                sentence_corrected = re.sub(' +', ' '," ".join(text.replace("\n"," ") for text in sentence[:actual_start].split(" ")))
                split_sentence_corrected = sentence_corrected.split(" ")
                new_sentence = []
                for item in split_sentence_corrected:
                    new_first = re.findall(r"[\w]+|[\"\'*.,!?;]", item)
                    if (len(new_first) == 3) and ("'" in new_first):
                        if (new_first[1] == "'"):
                            new_first = [new_first[0],new_first[1] + new_first[2]]
                    new_sentence.extend(new_first)
                #print(new_sentence)
                    
                if len(new_sentence) == index:
                    return actual_start
                if flag and (len(new_sentence) == index - 1) :
                    actual_start += 1
                    return actual_start
                    
                
                n += 1
                
        m += 1
        
        
        
# Function to get starting offset in case of L

def get_end(sentence,index,to_find):
    
    flag = False
    
    split_sentence = sentence.split(" ")
    
    m = 0
    n = 0
    
    while m < len(split_sentence):
        #print(m)
        if to_find in split_sentence[m]:

            n = 0
            while n < len(sentence):
        
                start = m
                #print(sentence)
                #print(n)
                #print(split_sentence[start])
                #print(sentence.find(split_sentence[start],n))
                actual_start = sentence.find(split_sentence[start],n)
                
                while sentence[actual_start:actual_start + len(to_find)] != to_find:
                    actual_start += 1
                

                
                sentence_corrected = re.sub(' +', ' '," ".join(text.replace("\n"," ") for text in sentence[:actual_start].split(" ")))
                split_sentence_corrected = sentence_corrected.split(" ")
                new_sentence = []
                for item in split_sentence_corrected:
                    new_first = re.findall(r"[\w]+|[\"\'*.,!?;]", item)
                    if (len(new_first) == 3) and ("'" in new_first):
                        if (new_first[1] == "'"):
                            new_first = [new_first[0],new_first[1] + new_first[2]]
                    new_sentence.extend(new_first)
                    
                    
                if len(new_sentence) == index:
                    actual_start += len(to_find) - 1
                    if (sentence[actual_start:actual_start + 2] == "'s") or (sentence[actual_start:actual_start + 2] == "'t"):
                        actual_start += 2
                        
                    return actual_start
                
                n += 1
                
        m += 1
        
        #x = len(sentence) -1
        #return x
        

# Function to get starting and endind offset in case of U
        
def get_start_end(sentence,index,to_find):
    flag = False
    
    split_sentence = sentence.split(" ")
    
    m = 0
    n = 0
    #print("Hola")
    while m < len(split_sentence):
        #print(m)
        if to_find in split_sentence[m]:

            n = 0
            while n < len(sentence):
        
                start = m
                #print(sentence)
                #print(n)
                #print(split_sentence[start])
                #print(sentence.find(split_sentence[start],n))
                #print(sentence)
                actual_start = sentence.find(split_sentence[start],n)
                #print(actual_start)
                while sentence[actual_start:actual_start + len(to_find)] != to_find:
                    actual_start += 1
                    
                starting = actual_start
                
                if sentence[actual_start] == "'":
                    flag = True
                    
                
                sentence_corrected = re.sub(' +', ' '," ".join(text.replace("\n"," ") for text in sentence[:actual_start].split(" ")))
                #print(sentence_corrected)
                split_sentence_corrected = sentence_corrected.split(" ")
                new_sentence = []
                for item in split_sentence_corrected:
                    new_first = re.findall(r"[\w]+|[\"\'*.,!?;]", item)
                    if (len(new_first) == 3) and ("'" in new_first):
                        if (new_first[1] == "'"):
                            new_first = [new_first[0],new_first[1] + new_first[2]]
                    new_sentence.extend(new_first)
                    #if flag:
                #print(new_sentence)
                    
                    
                #print(to_find)
                #print(new_sentence)
                #print(index)
                if len(new_sentence) == index:
                    #print("Hola")
                    actual_start += len(to_find) - 1
                    #print(starting)
                    #print(actual_start)
                    if (sentence[actual_start:actual_start + 2] == "'s") or (sentence[actual_start:actual_start + 2] == "'t"):
                        actual_start += 2
                    return starting, actual_start
                    
                elif flag and (len(new_sentence) == index - 1):
                    starting += 1
                    actual_start += 1
                    actual_start += len(to_find) - 1
                    
                    if (sentence[actual_start:actual_start + 2] == "'s") or (sentence[actual_start:actual_start + 2] == "'t"):
                        actual_start += 2
                        
                    
                        
                    return starting, actual_start
                
                n += 1
                
        m += 1
        

        


In [ ]:
# Call the above function to get starting and ending offsets

import re

all_predicted = []
for j in range(len(answers_pred)):
    i = 0
    sentence_predicted_span = []
    
    if (j == 1693) :
        all_predicted.append(sentence_predicted_span)
        continue
        
    while i < len(answers_token[j]):
        sentence_predicted_one_span = []
        
        if answers_pred[j][i] == "B":
            start = get_start(train["text"].iloc[j],i,answers_token[j][i])
            while answers_pred[j][i] != "L":
                i+=1
            if answers_pred[j][i] == "L":
                end = get_end(train["text"].iloc[j],i,answers_token[j][i])
                if end == None:
                    end = len(train["text"].iloc[j]) - 2
            sentence_predicted_one_span.append(start)
            sentence_predicted_one_span.append(end)
            

        elif answers_pred[j][i] == "U":            
            start,end = get_start_end(train["text"].iloc[j],i,answers_token[j][i])
            sentence_predicted_one_span.append(start)
            sentence_predicted_one_span.append(end)
            
        if len(sentence_predicted_one_span):
            sentence_predicted_span.append(sentence_predicted_one_span)

        i += 1
    all_predicted.append(sentence_predicted_span)

In [ ]:
# Expand starting and ending offsets into list of offsets for submission

for i in range(len(all_predicted)):
    final_answer = []
    for j in range(len(all_predicted[i])):
        answer = [number for number in range(all_predicted[i][j][0],all_predicted[i][j][1] + 1)]
        final_answer.extend(answer)
    all_predicted[i] = final_answer

In [ ]:
# Write into a file

with open("results/Dev/spans-pred-bert-io.txt", "w") as out:
    for uid, text_scores in enumerate(all_predicted):
        out.write(f"{str(uid)}\t{str(text_scores)}\n")